## Reshape

In [1]:
import torch
import netron

class JustReshape(torch.nn.Module):
    def __init__(self):
        super(JustReshape, self).__init__()

    def forward(self, x):
        return x.view((x.shape[0], x.shape[1], x.shape[3], x.shape[2]))

net = JustReshape()
model_name = 'just_reshape.onnx'
dummy_input = torch.randn(2, 3, 4, 59)
torch.onnx.export(net, dummy_input, model_name, input_names=['input'], output_names=['output'], do_constant_folding=False)
netron.start(f'{model_name}')

Serving 'just_reshape.onnx' at http://localhost:8080


('localhost', 8080)

In [2]:
model_name = 'just_reshape_sim.onnx'
torch.onnx.export(net, dummy_input, model_name, input_names=['input'], output_names=['output'], do_constant_folding=True)
netron.start(f'{model_name}')

Serving 'just_reshape_sim.onnx' at http://localhost:8081


('localhost', 8081)

In [3]:
!onnxsim "just_reshape.onnx" "just_reshape_sim2.onnx"

Simplifying...
Finish! Here is the difference:
┏━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃            ┃ Original Model ┃ Simplified Model ┃
┡━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ Concat     │ 1              │ 0                │
│ Constant   │ 8              │ 1                │
│ Reshape    │ 1              │ 1                │
│ Unsqueeze  │ 4              │ 0                │
│ Model Size │ 1.2KiB         │ 209.0B           │
└────────────┴────────────────┴──────────────────┘


In [4]:
netron.start('just_reshape_sim2.onnx')

Serving 'just_reshape_sim2.onnx' at http://localhost:21165


('localhost', 21165)

## ConvBlock

In [5]:
conv_block = torch.nn.Sequential(torch.nn.Conv2d(3, 16, 3, padding=1), torch.nn.BatchNorm2d(16), torch.nn.ReLU())
model_name = 'conv.onnx'
dummy_input = torch.randn(2, 3, 59, 59)
torch.onnx.export(conv_block, dummy_input, model_name, input_names=['input'], output_names=['output'], do_constant_folding=False)
netron.start(f'{model_name}')

Serving 'conv.onnx' at http://localhost:22998


('localhost', 22998)

In [6]:
model_name = 'conv_sim.onnx'
torch.onnx.export(conv_block, dummy_input, model_name, input_names=['input'], output_names=['output'], do_constant_folding=True)
netron.start(f'{model_name}')

Serving 'conv_sim.onnx' at http://localhost:23281


('localhost', 23281)

In [7]:
!onnxsim "conv.onnx" "conv_sim2.onnx"

Simplifying...
Finish! Here is the difference:
┏━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃                    ┃ Original Model ┃ Simplified Model ┃
┡━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ BatchNormalization │ 1              │ 1                │
│ Constant           │ 4              │ 4                │
│ Conv               │ 1              │ 1                │
│ Identity           │ 2              │ 0                │
│ Relu               │ 1              │ 1                │
│ Model Size         │ 2.5KiB         │ 2.5KiB           │
└────────────────────┴────────────────┴──────────────────┘


In [8]:
netron.start('conv_sim2.onnx')

Serving 'conv_sim2.onnx' at http://localhost:23845


('localhost', 23845)